# TP 1 : pré-traitement des données et visualisation

Nous allons travailler sur le jeu de données `gro.csv` pour faire du **credit scoring**, qui a été proposé il y a quelques années sur un site de data challenge. C'est un vrai de jeu qui contient des données avec différents types de variable, des valeurs manquantes, etc. Nous devons donc faire un gros travail de préparation de données.
Les données contiennent les colonnes suivantes :

| Nom de la colonne    | Description |
|:---------------------|:------------|
| BirthDate            | Date of birth of the client |
| Customer_Open_Date   | Creation date of the client's first account at the bank |
| Customer_Type        | Type of client (existing / new) | 
| Educational_Level    | Highest diploma |
| Id_Customer          | Id of the client |
| Marital_Status       | Family situation |
| Nb_Of_Products       | Number of products held by the client |
| Net_Annual_Income    | Annual revenue |
| Number_Of_Dependant  | Number of dependents |
| P_Client             | Non-disclosed feature |
| Prod_Category        | Product category |
| Prod_Closed_Date     | Closing date of the last product |
| Prod_Decision_Date   | Decision date of the last agreement for a financing product |
| Prod_Sub_Category    | Sub-category of the product |
| Source               | Financing source (Branch or Sales) |
| Type_Of_Residence    | Residential situation |
| Y                    | Credit was granted (yes / no) |
| Years_At_Business    | Number of year at the current job position |
| Years_At_Residence   | Number of year at the current housing |

On commence par importer les données via `pd.read_csv` en changeant le séparateur par défaut.

In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle as pkl
%matplotlib inline

sns.set_context("notebook", font_scale=1.2)


In [41]:
gro_dtypes = {
    'Years_At_Residence': np.int,
    'Net_Annual_Income' : np.float,
    'Years_At_Business': np.float,
    'Number_Of_Dependant': np.float,
    'Nb_Of_Products': np.int,
    'Customer_Type': 'category',
    'P_Client': 'category',
    'Educational_Level': 'category',
    'Marital_Status': 'category',
    'Prod_Sub_Category': 'category',
    'Source': 'category',
    'Type_Of_Residence': 'category',
    'Prod_Category': 'category',
}

df = pd.read_csv(
    # Filename
    "gro.csv",
    # Column separator
    sep=';',
    # Decimal separator
    decimal=',',
    # Range of the columns to keep (remove the last three ones)
    usecols=range(19),
    # Which columns should be parsed as dates
    parse_dates=['BirthDate', 'Customer_Open_Date', 'Prod_Decision_Date', 
                 'Prod_Closed_Date'],
    # Specify some dtypes
    dtype=gro_dtypes
)

In [42]:
df.head(n=5)

,Id_Customer,Y,Customer_Type,BirthDate,Customer_Open_Date,P_Client,Educational_Level,Marital_Status,Number_Of_Dependant,Years_At_Residence,Net_Annual_Income,Years_At_Business,Prod_Sub_Category,Prod_Decision_Date,Source,Type_Of_Residence,Nb_Of_Products,Prod_Closed_Date,Prod_Category
0,6714,0,Non Existing Client,1971-02-09,2012-01-18,NP_Client,University,Married,3.0,10,36.0,3.0,C,2012-01-23,Sales,Owned,1,NaT,B
1,7440,0,Non Existing Client,1977-07-08,2012-02-13,NP_Client,University,Married,3.0,1,36.0,1.0,C,2012-02-14,Sales,Owned,1,NaT,B
2,573,0,Existing Client,1974-06-13,2009-04-02,P_Client,University,Married,0.0,12,18.0,2.0,C,2011-06-30,Sales,Parents,1,NaT,G
3,9194,0,Non Existing Client,1973-07-11,2012-03-04,NP_Client,University,Married,2.0,10,36.0,1.0,C,2012-04-04,Sales,Owned,1,NaT,B
4,3016,1,Existing Client,1982-08-07,2011-08-25,NP_Client,University,Married,3.0,3,36.0,1.0,C,2011-07-09,Sales,New rent,1,2012-12-31,L


In [43]:
df.shape

(6725, 19)

#### Les données manquantes

On repère dans quelles colonnes sont les valeurs maquantes.

In [44]:
df.isnull().sum()

Id_Customer               0
Y                         0
Customer_Type             0
BirthDate                 0
Customer_Open_Date        0
P_Client                  0
Educational_Level         0
Marital_Status            0
Number_Of_Dependant       2
Years_At_Residence        0
Net_Annual_Income         3
Years_At_Business         4
Prod_Sub_Category         0
Prod_Decision_Date        0
Source                    0
Type_Of_Residence         0
Nb_Of_Products            0
Prod_Closed_Date       5291
Prod_Category             0
dtype: int64

C'est sur la colonne `Prod_Closed_Date` que se trouvent le plus grand nombre de valeurs manquantes, une solution est de la retirer. On peut également créer une nouvelle variable qui indique si `Prod_Closed_Date`est manquant ou non

In [45]:
df[['Prod_Closed_Date']].isnull()

,Prod_Closed_Date
0,True
1,True
2,True
3,True
4,False
...,...
6720,True
6721,False
6722,False
6723,True


In [46]:
df[['Prod_Closed_Date']].head(5)

,Prod_Closed_Date
0,NaT
1,NaT
2,NaT
3,NaT
4,2012-12-31


In [47]:
df['Prod_Closed_Date_missing'] = df[['Prod_Closed_Date']].isnull()
df.drop(['Prod_Closed_Date'], axis="columns", inplace=True)


In [48]:
df.shape

(6725, 19)

On regarde où les données contiennent encore des valeurs manquantes

In [49]:
df.isnull().sum()

Id_Customer                 0
Y                           0
Customer_Type               0
BirthDate                   0
Customer_Open_Date          0
P_Client                    0
Educational_Level           0
Marital_Status              0
Number_Of_Dependant         2
Years_At_Residence          0
Net_Annual_Income           3
Years_At_Business           4
Prod_Sub_Category           0
Prod_Decision_Date          0
Source                      0
Type_Of_Residence           0
Nb_Of_Products              0
Prod_Category               0
Prod_Closed_Date_missing    0
dtype: int64

Pour `Number_Of_Dependant`, `Net_Annual_Income`, `Years_At_Business`, on remplace par la valeur médiane, on va d'abord seuiller la variable `Net_Annual_Income`

In [50]:
df.loc[df['Net_Annual_Income'] > 200, 'Net_Annual_Income'] = 200

In [51]:


df[['Number_Of_Dependant', 'Net_Annual_Income', 'Years_At_Business']] = \
    df[['Number_Of_Dependant', 'Net_Annual_Income', 'Years_At_Business']].fillna(value = 
                                                                                 df[['Number_Of_Dependant', 'Net_Annual_Income', 'Years_At_Business']].median())


In [52]:
df.isnull().sum()

Id_Customer                 0
Y                           0
Customer_Type               0
BirthDate                   0
Customer_Open_Date          0
P_Client                    0
Educational_Level           0
Marital_Status              0
Number_Of_Dependant         0
Years_At_Residence          0
Net_Annual_Income           0
Years_At_Business           0
Prod_Sub_Category           0
Prod_Decision_Date          0
Source                      0
Type_Of_Residence           0
Nb_Of_Products              0
Prod_Category               0
Prod_Closed_Date_missing    0
dtype: int64

In [53]:
# Now we save the cleaned dataset into a CSV file
df.to_csv("gro_cleaned_nomissing.csv")

In [54]:
ls -al gro_*

-rw-r--r--@ 1 agatheguilloux  staff   850609  9 déc 10:27 gro_cleaned.csv
-rw-r--r--@ 1 agatheguilloux  staff   648142  4 jan 16:23 gro_cleaned.pkl
-rw-r--r--@ 1 agatheguilloux  staff   886755  8 jan 09:08 gro_cleaned_nomissing.csv
-rw-r--r--@ 1 agatheguilloux  staff  1864386  4 jan 16:25 gro_prep_array.pkl
-rw-r--r--@ 1 agatheguilloux  staff  1917707  4 jan 16:25 gro_prep_df.pkl


# Préparation finale des données

On prépare des listes avec les noms des features catégorielles, dates, et continues

In [55]:

cnt_featnames = [
    'Years_At_Residence',
    'Net_Annual_Income',
    'Years_At_Business',
    'Number_Of_Dependant'
]

cat_featnames = [
    'Customer_Type',
    'P_Client',
    'Educational_Level',
    'Marital_Status',
    'Prod_Sub_Category',
    'Source',
    'Type_Of_Residence',
    'Prod_Category',
    'Nb_Of_Products',
    'Prod_Closed_Date_missing'
]

date_featnames = [
    'BirthDate',
    'Customer_Open_Date',
    'Prod_Decision_Date'
    
]

In [66]:
np.unique(df[['Nb_Of_Products']])

array([1, 2, 3])

## On crée la matrice de design avec sklearn

In [56]:
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from pandas import Timestamp

# First we create a scikit-learn encoder that computes 
#  the age in days of columns containing dates
class AgeEncoder(TransformerMixin, BaseEstimator):
    def fit(self, X, y=None):
        self.today = Timestamp.today()
        return self

    def transform(self, X):
        return X.apply(lambda x: (x - self.today).dt.days, axis=0)

# Centers and reduces (variance=1) columns
standard_scaler = StandardScaler()

# One-hot encode, similar to pd.get_dummies
one_hot_encoder = OneHotEncoder(drop='first', sparse=False)

# A pipeline that first computes age, and standardizes it
scaled_age_encoder = Pipeline([
    ('age', AgeEncoder()),
    ('scaling', StandardScaler())
])

# Let's combine all these transformations
transformer = ColumnTransformer([
    ('standard_scaling', standard_scaler, cnt_featnames),
    ('one_hot_encoding', one_hot_encoder, cat_featnames),
    ('dates_age_scaled', scaled_age_encoder, date_featnames)
])

gro_prep = transformer.fit_transform(df)
gro_prep.shape

(6725, 38)

Nous avons perdu le nom des variables, ce n'est pas grave pour de l'apprentissage pur. C'est une chose qui manque sur sklearn, on peut s'en sortir avec pandas mais ce n'est pas très élégant.

In [80]:
df_gro_prep['Y'] = df['Y']
df_gro_prep.to_pickle("gro_prep_df_nomissing.pkl")
df_gro_prep.to_csv("gro_prep_df_nomissing.csv")

TypeError: to_pickle() got an unexpected keyword argument 'index'